# High-level protein programming language tutorial

In this notebook, we will walk through creating a simple program encoding a symmetric protein and running an optimization run.
At the end of the notebook, we will also link to examples of more complex design programs.

We design proteins guided by [ESMFold](https://github.com/facebookresearch/esm#esmfold), so let's get set up by loading the ESMFold model.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from language import EsmFoldv1

folding_callback = EsmFoldv1()
folding_callback.load(device="cuda:0")

Now let's write a program.
Our high-level programming language is organized by a *syntax tree*, where *leaf nodes* correspond to protein sequence segments. To begin, let's create a leaf node defined on a fixed-length sequence segment with 25 amino acid residues:

In [3]:
from language import FixedLengthSequenceSegment
from language import ProgramNode

protomer = FixedLengthSequenceSegment(25)
def make_protomer_node():
    return ProgramNode(sequence_segment=protomer)

For this tutorial, we will make a symmetric protein by repeating the above protomer sequence $N$ times, where $N$ is the desired rotational symmetry.

Our programming language enables *internal nodes* in the syntax tree that let us hierarchically control multiple child nodes. To program a protein with 3-fold symmetry, let's create an internal node with $N = 3$ children.

In [4]:
N = 3
_node = ProgramNode(
    children=[make_protomer_node() for _ in range(N)],
)

To complete our programming language, we require a set of *constraints*.
An important set of constraints to apply to all proteins is to maximize ESMFold's confidence in the model structure, which we quantify by the pTM and mean pLDDT scores.
We also want to constrain the protein to have rotational symmetry.

Adding those three constraints (on pTM, pLDDT, and symmetry) completes the simple program for a symmetric protein.
We can also weight different constraints differently; for this tutorial, we will double the weight on the symmetry constraint.
The constraints are compiled into a single *energy function* (right now, via a linear combination of constraint functions), which we will use to guide the optimization algorithm

The code below will add the constraints and weights, compile the program, and print out a summary of the energy function.

In [6]:
from language import MaximizePTM, MaximizePLDDT, SymmetryRing

# Define the program.
program = ProgramNode(
    energy_function_terms=[MaximizePTM(), MaximizePLDDT(), SymmetryRing()],
    energy_function_weights=[1.0, 1.0, 1.0],
    children=[make_protomer_node() for _ in range(N)],
)

# Set up the program.
sequence, residue_indices = program.get_sequence_and_set_residue_index_ranges()

# Compute and print the energy function.
energy_terms = program.get_energy_term_functions()
folding_output = folding_callback.fold(sequence, residue_indices)
for name, weight, energy_fn in energy_terms:
    print(f"{name} = {weight:.1f} * {energy_fn(folding_output):.2f}")

root:MaximizePTM = 1.0 * 0.84
root:MaximizePLDDT = 1.0 * 0.65
root:SymmetryRing = 1.0 * 0.57


We are now ready to use this program to guide an optimization loop to design a protein sequence and structure.
We currently perform simulated annealing of an MCMC optimization algorithm to do design.
The code below will set up the simulated annealing algorithm, including the relevant parameters, and run the optimization loop. 

In [ ]:
from language import run_simulated_annealing

optimized_program = run_simulated_annealing(
    program=program,
    initial_temperature=1.0,
    annealing_rate=0.97,
    total_num_steps=10_000,
    folding_callback=folding_callback,
    display_progress=True,
)
print("Final sequence = {}".format(optimized_program.get_sequence_and_set_residue_index_ranges()[0]))

Output()

/private/home/brianhie/protein-bert-model/examples/protein-programming-language/language/optimize.py:56: 
RuntimeWarning: overflow encountered in exp
  np.exp(energy_differential / temperature),

This program provides a minimal example of symmetric protein design, but we can also design much more complex proteins, as described in [our paper](https://www.biorxiv.org/content/10.1101/2022.12.21.521526v1).

We provide example programs corresponding to our paper figures, as described in the table below:

| Design task                                 | Figure in paper | Program file                                                                |
|:--------------------------------------------|:---------------|:----------------------------------------------------------------------------|
| Free hallucination                          | Figure 2A       | [free_hallucination.py](programs/free_hallucination.py)                     |
| Fixed backbone design                       | Figure 2D       | [fixed_backbone.py](programs/fixed_backbone.py)                             |
| Secondary structure design                  | Figure 2G       | [secondary_structure.py](programs/secondary_structure.py)                   |
| Functional site scaffolding                 | Figure 2H       | [functional_site_scaffolding.py](programs/functional_site_scaffolding.py)   |
| Symmetric monomer design                    | Figure 3A       | [symmetric_monomer.py](programs/symmetric_monomer.py)                       |
| Two-level symmetric<br>homo-oligomer design | Figure 4A       | [symmetric_two_level_multimer.py](programs/symmetric_two_level_multimer.py) |
| Symmetric binding site<br>scaffolding       | Figure 5A       | [symmetric_binding.py](programs/symmetric_binding.py)                       |

These can easily be imported and run.
For example, the following code imports a symmetric binding site scaffolding program and computes the energy.

In [ ]:
from programs.symmetric_binding import symmetric_binding_il10

program = symmetric_binding_il10()

sequence, residue_indices = program.get_sequence_and_set_residue_index_ranges()

folding_output = folding_callback.fold(sequence, residue_indices)

energy_terms = program.get_energy_term_functions()
for name, weight, energy_fn in energy_terms:
    print(f"{name} = {weight:.1f} * {energy_fn(folding_output):.2f}")